In [1]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.cluster import KMeans
import numpy as np

In [2]:
df = pd.read_csv('../data/invoices.csv')


In [3]:
df.head()

,id,Amount Due,Closed,Currency,Customer,Date (UTC),Due Date (UTC),Ending Balance,Forgiven,Paid,...,Subtotal,Total Discount Amount,Applied Coupons,Tax,Tax Percent,Total,Amount Paid,Status,Exclusive Tax Amount,Inclusive Tax Amount
0,in_1Qx3plAFeGZf0UoOd9f9bW47,"0,00",False,eur,cus_NHFLrvkCpBFCs7,2025-02-27 9:56,NaN,NaN,False,False,...,"0,00","0,00",NaN,"0,00",20.0,"0,00","0,00",draft,"0,00","0,00"
1,in_1Qx3phAFeGZf0UoOy7Khkj9i,"0,00",False,eur,cus_LSl85Ix2Ek965l,2025-02-27 9:56,NaN,NaN,False,False,...,"0,00","0,00",NaN,"0,00",19.0,"0,00","0,00",draft,"0,00","0,00"
2,in_1Qx3XqAFeGZf0UoONAAW2muL,"104,40",False,eur,cus_PQFFOsNVNtuBN9,2025-02-27 9:38,2025-03-02 9:37,NaN,False,False,...,"87,00","0,00",NaN,"17,40",20.0,"104,40","0,00",draft,"17,40","0,00"
3,in_1Qx3XqAFeGZf0UoOl79vERDa,"39,00",False,eur,cus_Q7apBCt36xeGbK,2025-02-27 9:38,NaN,NaN,False,False,...,"39,00","0,00",NaN,NaN,NaN,"39,00","0,00",draft,NaN,NaN
4,in_1Qx3WCAFeGZf0UoOcbJhPyor,"82,80",False,eur,cus_MU4dOalEEat1Ah,2025-02-27 9:36,NaN,NaN,False,False,...,"69,00","0,00",NaN,"13,80",20.0,"82,80","0,00",draft,"13,80","0,00"


In [4]:
df.describe()

,Tax Percent
count,3078.000000
mean,20.725276
std,2.285339
min,7.700000
25%,19.000000
50%,20.000000
75%,21.000000
max,27.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8075 entries, 0 to 8074
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    8075 non-null   object 
 1   Amount Due                            8075 non-null   object 
 2   Closed                                8075 non-null   bool   
 3   Currency                              8075 non-null   object 
 4   Customer                              8075 non-null   object 
 5   Date (UTC)                            8075 non-null   object 
 6   Due Date (UTC)                        224 non-null    object 
 7   Ending Balance                        8037 non-null   object 
 8   Forgiven                              8075 non-null   bool   
 9   Paid                                  8075 non-null   bool   
 10  Paid At (UTC)                         7765 non-null   object 
 11  Marked Uncollecti

In [6]:
date_columns = [
    'Date (UTC)', 'Due Date (UTC)', 'Finalized At (UTC)', 
    'Marked Uncollectible At (UTC)', 'Voided At (UTC)',
    'Minimum Line Item Period Start (UTC)', 'Maximum Line Item Period End (UTC)',
    'Period Start (UTC)', 'Period End (UTC)', 'Paid At (UTC)'
]

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

numeric_columns = [
    'Amount Due', 'Ending Balance', 'Starting Balance', 'Subtotal',
    'Total Discount Amount', 'Tax', 'Total', 'Amount Paid',
    'Exclusive Tax Amount', 'Inclusive Tax Amount'
]

for col in numeric_columns:
    df[col] = df[col].str.replace(',', '.').astype(float)

df['Tax Percent'] = df['Tax Percent'].astype(float)

categorical_columns = ['Currency', 'Status']
for col in categorical_columns:
    df[col] = df[col].astype('category')

df.columns = df.columns.str.lower()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8075 entries, 0 to 8074
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    8075 non-null   object        
 1   amount due                            8075 non-null   float64       
 2   closed                                8075 non-null   bool          
 3   currency                              8075 non-null   category      
 4   customer                              8075 non-null   object        
 5   date (utc)                            8075 non-null   datetime64[ns]
 6   due date (utc)                        224 non-null    datetime64[ns]
 7   ending balance                        8037 non-null   float64       
 8   forgiven                              8075 non-null   bool          
 9   paid                                  8075 non-null   bool          
 10  

In [8]:
df.shape

(8075, 30)

In [9]:
df.currency.unique()

['eur', 'usd']
Categories (2, object): ['eur', 'usd']

In [10]:
df[df['amount paid'] != 0][['currency', 'amount due', 'amount paid']].head()

,currency,amount due,amount paid
7,eur,22.50,22.50
14,eur,79.00,79.00
18,usd,9.00,9.00
21,eur,78.00,78.00
22,eur,37.58,37.58


In [11]:
df['subscription'].nunique()

4620

In [12]:
df['subscription'].head()

0    sub_1Mg3RXAFeGZf0UoOk6tgmHXy
1    sub_1LmaWWAFeGZf0UoO6OEZ7teq
2    sub_1QPhgOAFeGZf0UoOf7uRPvxM
3    sub_1Q3abzAFeGZf0UoO5UPNkeq7
4    sub_1OH1EMAFeGZf0UoO9at6xQkP
Name: subscription, dtype: object

In [13]:
df['subscription'].isna().sum()

np.int64(39)

In [14]:
df[df['subscription'].isna()].head()

,id,amount due,closed,currency,customer,date (utc),due date (utc),ending balance,forgiven,paid,...,subtotal,total discount amount,applied coupons,tax,tax percent,total,amount paid,status,exclusive tax amount,inclusive tax amount
7,in_1Qx3R0AFeGZf0UoOYSx37Ftb,22.5,True,eur,cus_PbPmpwRy3HrxgP,2025-02-27 09:31:00,2025-02-27 22:59:00,0.0,False,True,...,22.5,0.0,NaN,NaN,NaN,22.5,22.5,paid,NaN,NaN
18,in_1Qx2hmAFeGZf0UoOYD4eoPnB,9.0,True,usd,cus_PVdPFGa7V3RK3o,2025-02-27 08:44:00,2025-02-27 22:59:00,0.0,False,True,...,9.0,0.0,NaN,NaN,NaN,9.0,9.0,paid,NaN,NaN
27,in_1Qx1pRAFeGZf0UoOXawHgA3a,137.0,True,usd,cus_PVdPFGa7V3RK3o,2025-02-27 07:48:00,2025-02-27 22:59:00,0.0,True,False,...,137.0,0.0,NaN,NaN,NaN,137.0,0.0,uncollectible,NaN,NaN
256,in_1QwOhtAFeGZf0UoOSjYsHGRs,0.0,True,eur,cus_QKkoyiGneQSxxq,2025-02-25 14:01:00,1970-01-01 00:00:00,NaN,False,False,...,0.0,0.0,NaN,NaN,NaN,0.0,0.0,draft,NaN,NaN
320,in_1QwJVPAFeGZf0UoOGWkzGxBD,9.0,True,usd,cus_PVdPFGa7V3RK3o,2025-02-25 08:28:00,2025-02-25 22:59:00,0.0,False,True,...,9.0,0.0,NaN,NaN,NaN,9.0,9.0,paid,NaN,NaN


In [15]:
currency_counts = df.groupby('customer')['currency'].nunique()

multi_currency_customers = currency_counts[currency_counts > 1]

print("Customers with multiple currencies:")
print(multi_currency_customers)


Customers with multiple currencies:
customer
cus_Hgtp2cnDHmaKZd    2
cus_NGa2cqdacckhAF    2
cus_RHGc6HDDdE6sgP    2
cus_RU1Fvz6zIUCPod    2
Name: currency, dtype: int64


In [16]:
df['currency'].value_counts()

currency
eur    4690
usd    3385
Name: count, dtype: int64

In [17]:
df.groupby('currency')['total'].sum()

/var/folders/9q/fz04wycx2gq4qyhgmbnk9x_r0000gn/T/ipykernel_59481/2997685491.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('currency')['total'].sum()


currency
eur    508791.04
usd    331614.37
Name: total, dtype: float64

In [18]:
cust_df = pd.read_csv('../data/customers.csv')


In [19]:
cust_df.head()

,id,Created (UTC),Total Spend,Payment Count,Tax Location Recognized
0,cus_RqlO8Gz01CYRXm,2025-02-27 9:58,"0,00",0,False
1,cus_RqlL8NuTqGUuYi,2025-02-27 9:55,"0,00",0,False
2,cus_RqlERu33ao23ze,2025-02-27 9:48,"0,00",0,False
3,cus_RqlBBlHHmmwnT1,2025-02-27 9:45,"0,00",0,False
4,cus_Rqkr7zhFuvltT1,2025-02-27 9:26,"0,00",0,False


In [20]:
cust_df['Created (UTC)'] = pd.to_datetime(cust_df['Created (UTC)'])


In [21]:
cust_df['months_since_joined'] = ((pd.Timestamp.now() - cust_df['Created (UTC)']).dt.days / 30.44).round(0)


In [22]:
cust_df.head()

,id,Created (UTC),Total Spend,Payment Count,Tax Location Recognized,months_since_joined
0,cus_RqlO8Gz01CYRXm,2025-02-27 09:58:00,"0,00",0,False,1.0
1,cus_RqlL8NuTqGUuYi,2025-02-27 09:55:00,"0,00",0,False,1.0
2,cus_RqlERu33ao23ze,2025-02-27 09:48:00,"0,00",0,False,1.0
3,cus_RqlBBlHHmmwnT1,2025-02-27 09:45:00,"0,00",0,False,1.0
4,cus_Rqkr7zhFuvltT1,2025-02-27 09:26:00,"0,00",0,False,1.0


In [23]:
filtered_df = df.copy()
filtered_df = filtered_df[filtered_df['subscription'].notna()]
filtered_df = filtered_df[~filtered_df['forgiven']]



In [24]:
filtered_df.head()

,id,amount due,closed,currency,customer,date (utc),due date (utc),ending balance,forgiven,paid,...,subtotal,total discount amount,applied coupons,tax,tax percent,total,amount paid,status,exclusive tax amount,inclusive tax amount
0,in_1Qx3plAFeGZf0UoOd9f9bW47,0.0,False,eur,cus_NHFLrvkCpBFCs7,2025-02-27 09:56:00,NaT,NaN,False,False,...,0.0,0.0,NaN,0.0,20.0,0.0,0.0,draft,0.0,0.0
1,in_1Qx3phAFeGZf0UoOy7Khkj9i,0.0,False,eur,cus_LSl85Ix2Ek965l,2025-02-27 09:56:00,NaT,NaN,False,False,...,0.0,0.0,NaN,0.0,19.0,0.0,0.0,draft,0.0,0.0
2,in_1Qx3XqAFeGZf0UoONAAW2muL,104.4,False,eur,cus_PQFFOsNVNtuBN9,2025-02-27 09:38:00,2025-03-02 09:37:00,NaN,False,False,...,87.0,0.0,NaN,17.4,20.0,104.4,0.0,draft,17.4,0.0
3,in_1Qx3XqAFeGZf0UoOl79vERDa,39.0,False,eur,cus_Q7apBCt36xeGbK,2025-02-27 09:38:00,NaT,NaN,False,False,...,39.0,0.0,NaN,NaN,NaN,39.0,0.0,draft,NaN,NaN
4,in_1Qx3WCAFeGZf0UoOcbJhPyor,82.8,False,eur,cus_MU4dOalEEat1Ah,2025-02-27 09:36:00,NaT,NaN,False,False,...,69.0,0.0,NaN,13.8,20.0,82.8,0.0,draft,13.8,0.0


In [25]:
pivot_df = filtered_df.pivot_table(
    values='total',
    index='customer',
    columns='currency',
    aggfunc='sum',
    fill_value=0
)

/var/folders/9q/fz04wycx2gq4qyhgmbnk9x_r0000gn/T/ipykernel_59481/695849893.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = filtered_df.pivot_table(


In [26]:
pivot_df['grand_total'] = pivot_df.sum(axis=1)

In [27]:
pivot_df = pivot_df.sort_values('grand_total', ascending=False)

In [28]:
pivot_df.head()

currency,eur,usd,grand_total
customer,,,
cus_R4uIG2uftIlUlx,25000.00,0.0,25000.00
cus_HVt3QQArXUMKDv,21228.00,0.0,21228.00
cus_ONrcjxCBdtvpdd,0.00,15346.8,15346.80
cus_PM1exzslbBdyKL,10440.00,0.0,10440.00
cus_Iugqw6tkOVqBCY,7591.44,0.0,7591.44


In [29]:
pivot_df.describe()

currency,eur,usd,grand_total
count,4544.000000,4544.000000,4544.000000
mean,109.645374,65.332359,174.977733
std,607.199415,313.931047,673.052274
min,-112.990000,-52.570000,-112.990000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,78.000000
75%,78.000000,63.375000,156.000000
max,25000.000000,15346.800000,25000.000000


In [30]:
final_df = pivot_df.merge(
    cust_df[['id', 'months_since_joined']], 
    left_index=True, 
    right_on='id'
)

In [31]:
final_df = final_df.sort_values('grand_total', ascending=False)

In [32]:
final_df.head()

,eur,usd,grand_total,id,months_since_joined
7960,25000.00,0.0,25000.00,cus_R4uIG2uftIlUlx,5.0
35713,21228.00,0.0,21228.00,cus_HVt3QQArXUMKDv,57.0
24485,0.00,15346.8,15346.80,cus_ONrcjxCBdtvpdd,20.0
21692,10440.00,0.0,10440.00,cus_PM1exzslbBdyKL,15.0
34975,7591.44,0.0,7591.44,cus_Iugqw6tkOVqBCY,50.0


In [33]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=final_df[final_df['eur'] > 0]['eur'],
    name='EUR Distribution',
    opacity=0.7
))

fig.add_trace(go.Histogram(
    x=final_df[final_df['usd'] > 0]['usd'],
    name='USD Distribution',
    opacity=0.7
))

fig.update_layout(
    title='Distribution of Customer Totals by Currency',
    xaxis_title='Amount',
    yaxis_title='Count of Customers',
    barmode='overlay',
    showlegend=True
)

fig.show()

In [34]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=final_df[final_df['eur'] >= 0]['eur'],
    name='EUR Distribution',
    opacity=0.7,
    nbinsx=50
))

fig.add_trace(go.Histogram(
    x=final_df[final_df['usd'] >= 0]['usd'],
    name='USD Distribution',
    opacity=0.7,
    nbinsx=50
))

fig.update_layout(
    title='Distribution of Customer Totals by Currency (Positive Values Only)',
    xaxis_title='Amount',
    yaxis_title='Count of Customers',
    barmode='overlay',
    showlegend=True
)

fig.show()

In [35]:
def detect_outliers(series):
    upper_bound = series.quantile(0.99)
    outliers = series[series > upper_bound]
    return outliers, upper_bound

eur_outliers, eur_upper = detect_outliers(final_df['eur'])
print("EUR Outliers:")
print(f"99th percentile bound: {eur_upper:.2f}")
print(f"Number of outliers: {len(eur_outliers)}")


usd_outliers, usd_upper = detect_outliers(final_df['usd'])
print("\nUSD Outliers:")
print(f"99th percentile bound: {usd_upper:.2f}")
print(f"Number of outliers: {len(usd_outliers)}")


EUR Outliers:
99th percentile bound: 1228.54
Number of outliers: 46

USD Outliers:
99th percentile bound: 711.43
Number of outliers: 46


In [36]:
def create_clusters(series, n_clusters=5):
    zero_neg_mask = series <= 0
    positive_values = series[~zero_neg_mask].values.reshape(-1, 1)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    positive_clusters = kmeans.fit_predict(positive_values)
    
    centers = kmeans.cluster_centers_.flatten()
    sorted_centers_idx = np.argsort(centers)
    
    cluster_mapping = {old: new for new, old in enumerate(sorted_centers_idx)}
    
    positive_clusters = np.array([cluster_mapping[i] for i in positive_clusters])
    
    cluster_labels = pd.Series(index=series.index, dtype='object')
    cluster_labels[zero_neg_mask] = 'Zero/Negative'
    
    positive_indices = series[~zero_neg_mask].index
    for i, label in enumerate(positive_clusters):
        cluster_labels[positive_indices[i]] = f'Cluster {label + 1}'
    
    sorted_centers = centers[sorted_centers_idx]
    
    return cluster_labels, sorted_centers

In [37]:
eur_clusters, eur_centers = create_clusters(final_df['eur'])
print("EUR Clusters:")
print("\nCluster Centers:")
for i, center in enumerate(eur_centers):
    print(f"Cluster {i+1}: {center:.2f}")

print("\nCluster Distribution:")
print(eur_clusters.value_counts())

EUR Clusters:

Cluster Centers:
Cluster 1: 127.87
Cluster 2: 680.06
Cluster 3: 1990.17
Cluster 4: 6664.30
Cluster 5: 23114.00

Cluster Distribution:
Zero/Negative    2687
Cluster 1        1627
Cluster 2         187
Cluster 3          32
Cluster 4           8
Cluster 5           2
Name: count, dtype: int64


In [38]:
usd_clusters, usd_centers = create_clusters(final_df['usd'])
print("\nUSD Clusters:")
print("\nCluster Centers:")
for i, center in enumerate(usd_centers):
    print(f"Cluster {i+1}: {center:.2f}")

print("\nCluster Distribution:")
print(usd_clusters.value_counts())


USD Clusters:

Cluster Centers:
Cluster 1: 128.04
Cluster 2: 717.80
Cluster 3: 2167.39
Cluster 4: 4125.30
Cluster 5: 15346.80

Cluster Distribution:
Zero/Negative    3149
Cluster 1        1278
Cluster 2          96
Cluster 3          15
Cluster 4           4
Cluster 5           1
Name: count, dtype: int64


In [39]:
final_df['eur_cluster'] = eur_clusters
final_df['usd_cluster'] = usd_clusters

In [40]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=final_df[final_df['eur'] > 0]['eur'],
    y=final_df[final_df['eur'] > 0]['eur_cluster'],
    mode='markers',
    name='EUR Clusters',
    marker=dict(size=10)
))

fig.add_trace(go.Scatter(
    x=final_df[final_df['usd'] > 0]['usd'],
    y=final_df[final_df['usd'] > 0]['usd_cluster'],
    mode='markers',
    name='USD Clusters',
    marker=dict(size=10)
))

fig.update_layout(
    title='Customer Clusters by Currency (5 Clusters)',
    xaxis_title='Amount',
    yaxis_title='Cluster',
    showlegend=True
)

fig.show()

In [41]:
final_df[final_df['eur_cluster'] == 'Cluster 5']

,eur,usd,grand_total,id,months_since_joined,eur_cluster,usd_cluster
7960,25000.0,0.0,25000.0,cus_R4uIG2uftIlUlx,5.0,Cluster 5,Zero/Negative
35713,21228.0,0.0,21228.0,cus_HVt3QQArXUMKDv,57.0,Cluster 5,Zero/Negative


In [42]:
final_df[final_df['usd_cluster'] == 'Cluster 5']

,eur,usd,grand_total,id,months_since_joined,eur_cluster,usd_cluster
24485,0.0,15346.8,15346.8,cus_ONrcjxCBdtvpdd,20.0,Zero/Negative,Cluster 5


In [43]:
df.head()

,id,amount due,closed,currency,customer,date (utc),due date (utc),ending balance,forgiven,paid,...,subtotal,total discount amount,applied coupons,tax,tax percent,total,amount paid,status,exclusive tax amount,inclusive tax amount
0,in_1Qx3plAFeGZf0UoOd9f9bW47,0.0,False,eur,cus_NHFLrvkCpBFCs7,2025-02-27 09:56:00,NaT,NaN,False,False,...,0.0,0.0,NaN,0.0,20.0,0.0,0.0,draft,0.0,0.0
1,in_1Qx3phAFeGZf0UoOy7Khkj9i,0.0,False,eur,cus_LSl85Ix2Ek965l,2025-02-27 09:56:00,NaT,NaN,False,False,...,0.0,0.0,NaN,0.0,19.0,0.0,0.0,draft,0.0,0.0
2,in_1Qx3XqAFeGZf0UoONAAW2muL,104.4,False,eur,cus_PQFFOsNVNtuBN9,2025-02-27 09:38:00,2025-03-02 09:37:00,NaN,False,False,...,87.0,0.0,NaN,17.4,20.0,104.4,0.0,draft,17.4,0.0
3,in_1Qx3XqAFeGZf0UoOl79vERDa,39.0,False,eur,cus_Q7apBCt36xeGbK,2025-02-27 09:38:00,NaT,NaN,False,False,...,39.0,0.0,NaN,NaN,NaN,39.0,0.0,draft,NaN,NaN
4,in_1Qx3WCAFeGZf0UoOcbJhPyor,82.8,False,eur,cus_MU4dOalEEat1Ah,2025-02-27 09:36:00,NaT,NaN,False,False,...,69.0,0.0,NaN,13.8,20.0,82.8,0.0,draft,13.8,0.0


In [44]:
filtered_df.head()

,id,amount due,closed,currency,customer,date (utc),due date (utc),ending balance,forgiven,paid,...,subtotal,total discount amount,applied coupons,tax,tax percent,total,amount paid,status,exclusive tax amount,inclusive tax amount
0,in_1Qx3plAFeGZf0UoOd9f9bW47,0.0,False,eur,cus_NHFLrvkCpBFCs7,2025-02-27 09:56:00,NaT,NaN,False,False,...,0.0,0.0,NaN,0.0,20.0,0.0,0.0,draft,0.0,0.0
1,in_1Qx3phAFeGZf0UoOy7Khkj9i,0.0,False,eur,cus_LSl85Ix2Ek965l,2025-02-27 09:56:00,NaT,NaN,False,False,...,0.0,0.0,NaN,0.0,19.0,0.0,0.0,draft,0.0,0.0
2,in_1Qx3XqAFeGZf0UoONAAW2muL,104.4,False,eur,cus_PQFFOsNVNtuBN9,2025-02-27 09:38:00,2025-03-02 09:37:00,NaN,False,False,...,87.0,0.0,NaN,17.4,20.0,104.4,0.0,draft,17.4,0.0
3,in_1Qx3XqAFeGZf0UoOl79vERDa,39.0,False,eur,cus_Q7apBCt36xeGbK,2025-02-27 09:38:00,NaT,NaN,False,False,...,39.0,0.0,NaN,NaN,NaN,39.0,0.0,draft,NaN,NaN
4,in_1Qx3WCAFeGZf0UoOcbJhPyor,82.8,False,eur,cus_MU4dOalEEat1Ah,2025-02-27 09:36:00,NaT,NaN,False,False,...,69.0,0.0,NaN,13.8,20.0,82.8,0.0,draft,13.8,0.0


In [45]:
filtered_df['date (utc)'] = pd.to_datetime(filtered_df['date (utc)'])


In [46]:
filtered_df['month_year'] = filtered_df['date (utc)'].dt.to_period('M')
filtered_df['date'] = filtered_df['date (utc)'].dt.date


In [47]:
filtered_df[['date (utc)', 'month_year']].head()

,date (utc),month_year
0,2025-02-27 09:56:00,2025-02
1,2025-02-27 09:56:00,2025-02
2,2025-02-27 09:38:00,2025-02
3,2025-02-27 09:38:00,2025-02
4,2025-02-27 09:36:00,2025-02


In [48]:
daily_totals = filtered_df.groupby(['date', 'currency'])['total'].sum().unstack().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily_totals['date'],
    y=daily_totals['eur'],
    name='EUR Daily Total',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=daily_totals['date'],
    y=daily_totals['usd'],
    name='USD Daily Total',
    line=dict(color='red')
))

fig.update_layout(
    title='Daily Totals by Currency',
    xaxis_title='Date',
    yaxis_title='Total Amount',
    showlegend=True,
    hovermode='x unified'  
)

fig.show()

/var/folders/9q/fz04wycx2gq4qyhgmbnk9x_r0000gn/T/ipykernel_59481/2307437496.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [49]:
monthly_customers = filtered_df.groupby('month_year')['customer'].nunique().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_customers['month_year'].astype(str),
    y=monthly_customers['customer'],
    name='Unique Customers',
    line=dict(color='blue')
))

fig.update_layout(
    title='Number of Unique Customers by Month',
    xaxis_title='Month',
    yaxis_title='Number of Unique Customers',
    showlegend=True
)

fig.show()

In [50]:
filtered_df.head()

,id,amount due,closed,currency,customer,date (utc),due date (utc),ending balance,forgiven,paid,...,applied coupons,tax,tax percent,total,amount paid,status,exclusive tax amount,inclusive tax amount,month_year,date
0,in_1Qx3plAFeGZf0UoOd9f9bW47,0.0,False,eur,cus_NHFLrvkCpBFCs7,2025-02-27 09:56:00,NaT,NaN,False,False,...,NaN,0.0,20.0,0.0,0.0,draft,0.0,0.0,2025-02,2025-02-27
1,in_1Qx3phAFeGZf0UoOy7Khkj9i,0.0,False,eur,cus_LSl85Ix2Ek965l,2025-02-27 09:56:00,NaT,NaN,False,False,...,NaN,0.0,19.0,0.0,0.0,draft,0.0,0.0,2025-02,2025-02-27
2,in_1Qx3XqAFeGZf0UoONAAW2muL,104.4,False,eur,cus_PQFFOsNVNtuBN9,2025-02-27 09:38:00,2025-03-02 09:37:00,NaN,False,False,...,NaN,17.4,20.0,104.4,0.0,draft,17.4,0.0,2025-02,2025-02-27
3,in_1Qx3XqAFeGZf0UoOl79vERDa,39.0,False,eur,cus_Q7apBCt36xeGbK,2025-02-27 09:38:00,NaT,NaN,False,False,...,NaN,NaN,NaN,39.0,0.0,draft,NaN,NaN,2025-02,2025-02-27
4,in_1Qx3WCAFeGZf0UoOcbJhPyor,82.8,False,eur,cus_MU4dOalEEat1Ah,2025-02-27 09:36:00,NaT,NaN,False,False,...,NaN,13.8,20.0,82.8,0.0,draft,13.8,0.0,2025-02,2025-02-27


In [51]:
filtered_df['week'] = filtered_df['date (utc)'].dt.isocalendar().week
filtered_df['year'] = filtered_df['date (utc)'].dt.isocalendar().year

weekly_customers = filtered_df.groupby(['year', 'week'])['customer'].nunique().reset_index()

weekly_customers['week_year'] = weekly_customers['year'].astype(str) + '-W' + weekly_customers['week'].astype(str)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_customers['week_year'],
    y=weekly_customers['customer'],
    name='Unique Customers',
    line=dict(color='blue')
))

fig.update_layout(
    title='Number of Unique Customers by Week',
    xaxis_title='Week',
    yaxis_title='Number of Unique Customers',
    showlegend=True
)

fig.show()

In [52]:
cust_df.shape

(35748, 6)

In [53]:
cust_df.id.nunique()

35748

In [54]:
daily_totals.head()

currency,date,eur,usd
0,2024-12-31,0.00,15346.80
1,2025-01-01,4064.79,2207.70
2,2025-01-02,5765.18,3408.75
3,2025-01-03,10925.48,3391.46
4,2025-01-04,2707.89,11421.38


In [55]:
from prophets import ProphetModelBuilder

/Users/idreecekhan/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [56]:
daily_totals['date'] = pd.to_datetime(daily_totals['date'])

In [57]:
daily_totals.head()

currency,date,eur,usd
0,2024-12-31,0.00,15346.80
1,2025-01-01,4064.79,2207.70
2,2025-01-02,5765.18,3408.75
3,2025-01-03,10925.48,3391.46
4,2025-01-04,2707.89,11421.38


In [58]:
daily_totals.set_index('date', inplace=True)

In [59]:
eur_data = daily_totals[['eur']].copy()
eur_data.index.name = 'ds'
eur_data.columns = ['y']  

eur_model = ProphetModelBuilder(
    data=eur_data,
    target_column='y',
    yearly_seasonality=False,
    weekly_seasonality=True,
    daily_seasonality=True
)

13:59:59 - cmdstanpy - INFO - Chain [1] start processing
13:59:59 - cmdstanpy - INFO - Chain [1] done processing


In [60]:
usd_data = daily_totals[['usd']].copy()
usd_data.index.name = 'ds'
usd_data.columns = ['y']

usd_model = ProphetModelBuilder(
    data=usd_data,
    target_column='y',
    yearly_seasonality=False,
    weekly_seasonality=True,
    daily_seasonality=True
)

13:59:59 - cmdstanpy - INFO - Chain [1] start processing
13:59:59 - cmdstanpy - INFO - Chain [1] done processing


In [61]:
eur_forecast = eur_model.generate_forecasts(forecast_periods=30)
usd_forecast = usd_model.generate_forecasts(forecast_periods=30)

In [62]:
eur_conf_int = eur_model.generate_confidence_intervals(forecast_periods=30)
usd_conf_int = usd_model.generate_confidence_intervals(forecast_periods=30)

In [63]:
eur_fig = go.Figure()

eur_fig.add_trace(go.Scatter(
    x=eur_data.index,
    y=eur_data['y'],
    name='Actual',
    line=dict(color='blue')
))

future_mask = eur_forecast['ds'] > eur_data.index.max()
eur_fig.add_trace(go.Scatter(
    x=eur_forecast[future_mask]['ds'],
    y=eur_forecast[future_mask]['yhat'],
    name='Forecast',
    line=dict(color='red')
))

eur_fig.add_trace(go.Scatter(
    x=eur_conf_int[future_mask]['ds'],
    y=eur_conf_int[future_mask]['upper_bound'],
    fill=None,
    mode='lines',
    line_color='rgba(255,0,0,0)',
    name='Upper Bound'
))

eur_fig.add_trace(go.Scatter(
    x=eur_conf_int[future_mask]['ds'],
    y=eur_conf_int[future_mask]['lower_bound'],
    fill='tonexty',
    mode='lines',
    line_color='rgba(255,0,0,0)',
    name='Lower Bound'
))

eur_fig.update_layout(
    title='EUR Revenue Forecast',
    xaxis_title='Date',
    yaxis_title='Amount',
    hovermode='x unified'
)

eur_fig.show()




In [64]:
usd_fig = go.Figure()

usd_fig.add_trace(go.Scatter(
    x=usd_data.index,
    y=usd_data['y'],
    name='Actual',
    line=dict(color='blue')
))

future_mask = usd_forecast['ds'] > usd_data.index.max()
usd_fig.add_trace(go.Scatter(
    x=usd_forecast[future_mask]['ds'],
    y=usd_forecast[future_mask]['yhat'],
    name='Forecast',
    line=dict(color='red')
))

usd_fig.add_trace(go.Scatter(
    x=usd_conf_int[future_mask]['ds'],
    y=usd_conf_int[future_mask]['upper_bound'],
    fill=None,
    mode='lines',
    line_color='rgba(255,0,0,0)',
    name='Upper Bound'
))

usd_fig.add_trace(go.Scatter(
    x=usd_conf_int[future_mask]['ds'],
    y=usd_conf_int[future_mask]['lower_bound'],
    fill='tonexty',
    mode='lines',
    line_color='rgba(255,0,0,0)',
    name='Lower Bound'
))

usd_fig.update_layout(
    title='USD Revenue Forecast',
    xaxis_title='Date',
    yaxis_title='Amount',
    hovermode='x unified'
)

usd_fig.show()